# Setup

In [4]:
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec

2023-01-11 11:08:31.878284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 11:08:32.014401: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-11 11:08:32.014424: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-11 11:08:32.047231: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-11 11:08:32.797788: W tensorflow/stream_executor/platform/de

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd

In [7]:
!pwd

/home/verdy/code/TomVerdy/twitter_kaggle_challenge


# Aperçu de la data

In [8]:
data = pd.read_csv("Data/train.csv")

In [9]:
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [10]:
data.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [11]:
data["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
data_test = pd.read_csv("Data/test.csv")

In [13]:
data_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


# On encode la data

In [14]:
X = data["text"]

In [15]:
y = data["target"]

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

In [18]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [19]:
# Embed the training sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)

In [21]:
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [22]:
X_train_pad.shape

(5100, 200, 100)

In [23]:
y_train.shape

(5100,)

# Premier modèle

In [24]:
word2vec.wv.vector_size

100

In [25]:
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.metrics import *

In [45]:
rnn = Sequential([
    layers.Masking(mask_value=0),
    layers.LSTM(20, activation="tanh"),
    layers.Dense(10, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

rnn.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [53]:
from tensorflow.keras import callbacks
es = callbacks.EarlyStopping(patience=3,restore_best_weights=True)

rnn.fit(X_train_pad, y_train, epochs=10, batch_size=16, verbose=1)


Epoch 1/10
319/319 [==============================] - 32s 82ms/step - loss: 0.6575 - accuracy: 0.6108
Epoch 2/10
319/319 [==============================] - 28s 89ms/step - loss: 0.6493 - accuracy: 0.6212
Epoch 3/10
319/319 [==============================] - 26s 81ms/step - loss: 0.6454 - accuracy: 0.6265
Epoch 4/10
319/319 [==============================] - 28s 89ms/step - loss: 0.6440 - accuracy: 0.6290
Epoch 5/10
319/319 [==============================] - 29s 90ms/step - loss: 0.6392 - accuracy: 0.6351
Epoch 6/10
319/319 [==============================] - 28s 88ms/step - loss: 0.6379 - accuracy: 0.6380
Epoch 7/10
319/319 [==============================] - 28s 89ms/step - loss: 0.6338 - accuracy: 0.6390
Epoch 8/10
319/319 [==============================] - 29s 90ms/step - loss: 0.6279 - accuracy: 0.6433
Epoch 9/10
319/319 [==============================] - 28s 89ms/step - loss: 0.6247 - accuracy: 0.6447
Epoch 10/10
319/319 [==============================] - 28s 88ms/step - loss: 0.619

In [57]:
rnn.evaluate(X_test_pad, y_test)

79/79 [==============================] - 3s 25ms/step - loss: 0.6336 - accuracy: 0.6327


[0.6336041688919067, 0.6327099204063416]

# Modele si on clean les données ?

## data cleaning

In [27]:
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [31]:
import string
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [34]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/verdy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/verdy/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/verdy/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/verdy/nltk_data...


True

In [35]:
def preprocessing(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
        
    text = text.strip()
    text = text.lower() 
    text = ''.join(char for char in text if not char.isdigit())
    
    list_mots = text.split(" ")
    ls = []
    last = []
    for el in list_mots:
        ls.append(WordNetLemmatizer().lemmatize(WordNetLemmatizer().lemmatize(el, pos = "n"), pos = "v"))
    text = ' '.join(ls)
    
    text = text.replace("\n","")

    return text

In [36]:
data["clean_twitte"] = data["text"].map(preprocessing)

In [37]:
data

,id,keyword,location,text,target,clean_twitte
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deed be the reason of this earthquake may ...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all resident ask to shelter in place be be not...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order in c...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just get send this photo from ruby alaska a sm...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant crane hold a bridge collapse into ne...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,ariaahrary thetawniest the out of control wild...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m utckm s of volcano hawaii httptcozdtoydebj
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,police investigate after an ebike collide with...


In [38]:
X = data["clean_twitte"]

In [39]:
y = data["target"]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [41]:
word2vec = Word2Vec(sentences=X_train)

In [42]:
# Embed the training sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

In [43]:
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [44]:
X_train_pad.shape

(5100, 200, 100)

In [46]:
from tensorflow.keras import callbacks
es = callbacks.EarlyStopping(patience=3,restore_best_weights=True)

rnn.fit(X_train_pad, y_train, epochs=10, batch_size=16, verbose=1)


Epoch 1/10
319/319 [==============================] - 35s 98ms/step - loss: 0.6729 - accuracy: 0.5892
Epoch 2/10
319/319 [==============================] - 32s 99ms/step - loss: 0.6571 - accuracy: 0.6137
Epoch 3/10
319/319 [==============================] - 31s 97ms/step - loss: 0.6535 - accuracy: 0.6196
Epoch 4/10
319/319 [==============================] - 31s 98ms/step - loss: 0.6506 - accuracy: 0.6269
Epoch 5/10
319/319 [==============================] - 31s 97ms/step - loss: 0.6512 - accuracy: 0.6231
Epoch 6/10
319/319 [==============================] - 30s 96ms/step - loss: 0.6480 - accuracy: 0.6225
Epoch 7/10
319/319 [==============================] - 30s 94ms/step - loss: 0.6469 - accuracy: 0.6239
Epoch 8/10
319/319 [==============================] - 27s 85ms/step - loss: 0.6440 - accuracy: 0.6296
Epoch 9/10
319/319 [==============================] - 31s 96ms/step - loss: 0.6426 - accuracy: 0.6280
Epoch 10/10
319/319 [==============================] - 29s 92ms/step - loss: 0.640

In [47]:
rnn.evaluate(X_test_pad, y_test)

79/79 [==============================] - 3s 26ms/step - loss: 0.6501 - accuracy: 0.6220


[0.6500754952430725, 0.621965765953064]

**Pas besoin de préproc en Deep ?**

# Modele Conv1D (tokenize)

In [54]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
    
# We apply the tokenization to the train and test set
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)


X_train_pad = pad_sequences(X_train_token, maxlen=150, dtype='float32')
X_test_pad = pad_sequences(X_test_token, maxlen=150, dtype='float32')

vocab_size = len(tokenizer.word_index)
vocab_size

14630

In [59]:
def init_cnn_model(vocab_size):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size + 1, output_dim=100, mask_zero=True, input_length=150))
    model.add(layers.Conv1D(16, 3))
    model.add(layers.Flatten())
    model.add(layers.Dense(5,))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_cnn = init_cnn_model(vocab_size)

In [60]:
model_cnn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 100)          1463100   
                                                                 
 conv1d_1 (Conv1D)           (None, 148, 16)           4816      
                                                                 
 flatten_1 (Flatten)         (None, 2368)              0         
                                                                 
 dense_6 (Dense)             (None, 5)                 11845     
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,479,767
Trainable params: 1,479,767
Non-trainable params: 0
_________________________________________________________________


In [61]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_cnn.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
112/112 [==============================] - 2s 11ms/step - loss: 0.6327 - accuracy: 0.6373 - val_loss: 0.5016 - val_accuracy: 0.7641
Epoch 2/20
112/112 [==============================] - 1s 10ms/step - loss: 0.3273 - accuracy: 0.8611 - val_loss: 0.5008 - val_accuracy: 0.7699
Epoch 3/20
112/112 [==============================] - 1s 10ms/step - loss: 0.1013 - accuracy: 0.9675 - val_loss: 0.6036 - val_accuracy: 0.7680
Epoch 4/20
112/112 [==============================] - 1s 10ms/step - loss: 0.0380 - accuracy: 0.9885 - val_loss: 0.6961 - val_accuracy: 0.7379
Epoch 5/20
112/112 [==============================] - 1s 10ms/step - loss: 0.0225 - accuracy: 0.9933 - val_loss: 0.7191 - val_accuracy: 0.7667
Epoch 6/20
112/112 [==============================] - 1s 10ms/step - loss: 0.0210 - accuracy: 0.9947 - val_loss: 0.7152 - val_accuracy: 0.7608
Epoch 7/20
112/112 [==============================] - 1s 10ms/step - loss: 0.0179 - accuracy: 0.9947 - val_loss: 0.7722 - val_accuracy: 0.7771

## modele Conv1D (Word2vec)